In [ ]:
import os
# from itertools import cycle
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow import keras
# from tensorflow.keras import backend
# from tensorflow.keras import models
from tensorflow.keras.metrics import Recall, Precision, BinaryAccuracy, FalsePositives, \
                                    TruePositives, TrueNegatives, FalseNegatives, AUC, Metric
import sys
sys.path.insert(1, "../bin/")
from metrics_helper import MatthewsCorrelationCoefficient
from model_definition import load_compile_model
from test_zoonosis_model import load_test_data
# from tensorflow.python.keras.utils import metrics_utils
# from tensorflow.python.keras.utils.generic_utils import to_list
from tensorflow.keras.utils import plot_model
from sklearn.metrics import plot_roc_curve, precision_recall_curve, f1_score, roc_auc_score, \
                            roc_curve, matthews_corrcoef, confusion_matrix, auc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
# import hiplot as hip

# import plotly.graph_objects as go
# from mlxtend.plotting import plot_confusion_matrix
%matplotlib inline

# from tf_keras_vis.saliency import Saliency
# from tf_keras_vis.scorecam import Scorecam
# from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, enable=True) 

In [ ]:
physical_devices

In [ ]:
# main_dir = "/scratch2/users/rserage/"
main_dir = "/home/rserage/Documents/Masters Project/Docs"
test_dirs = os.listdir(main_dir)

test_dirs = [os.path.join(main_dir, fol) for fol in test_dirs]

In [ ]:
test_dirs = list(filter(lambda x: x.__contains__("Zoo"), test_dirs)) # "Data" for dataset

test_dirs

In [ ]:
# test_dirs = ["/scratch3/users/rserage/ZoonosisFull/"]
# model_dirs = ["/users/rserage/Zoon0PredV/notebooks/"]

In [ ]:
models = [os.path.join(path, 'model') for path in test_dirs]
test_data = [os.path.join(path, 'test') for path in test_dirs]

models

In [ ]:
main = pd.DataFrame()
PRS = dict()
ROCS = dict()

for model_, test_data in zip(models, test_data):
    model = load_compile_model(model_)
    metrics, PR, ROC = test_model(model, test_data)
    main = pd.concat([main, metrics], sort=False)
    PRS.update(PR)
    ROCS.update(ROC)

In [ ]:
ROCS

In [ ]:
main

In [ ]:
PRS

In [ ]:
ROC_curve(main, ROCS)

In [ ]:
PR_curve(main, PRS)

In [ ]:
model = load_compile_model("/users/rserage/Zoon0PredV/notebooks/model/")

In [ ]:
dot_img_file = "model_1.png"
plot_model(model, to_file=dot_img_file, show_layer_names=False, show_shapes=False)

In [ ]:
# data = [{
#     'epoch': idx,
#     'loss': history.history['loss'][idx],
#     'val_loss': history.history['val_loss'][idx],
#     'accuracy': history.history['accuracy'][idx],
#     'val_accuracy': history.history['val_accuracy'][idx],
#     'precision': history.history['precision'][idx],
#     'val_precision': history.history['val_precision'][idx],
#     'recall': history.history['recall'][idx],
#     'val_recall': history.history['val_recall'][idx]
# } 
#     for idx in range(0, 10)] 
df = pd.read_csv('logs.csv')
data = df.to_dict('records')

hip.Experiment.from_iterable(data).display()

In [ ]:
# test_data_iterator.reset()
for _ in range(5):
    img, label = test_data_iterator.next()
    print(img.shape)   #  (1,256,256,3)
    plt.figure(figsize=(12,6))
    plt.imshow(img[0], cmap=plt.cm.binary)
    # plt.show()
    # print(img[0])
    break

In [ ]:
# model = tf.keras.models.load_model('model', custom_objects={"MCC": MatthewsCorrelationCoefficient})
# model.summary()

In [ ]:
img_path = '/scratch3/users/rserage/ZoonosisData/test/human_false_A0A023NBJ1_Autoprotease.png'

# img_path = 'testing.png'

img = load_img(img_path, target_size=(128, 128), color_mode='grayscale')
img_tensor = img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

In [ ]:
img_tensor, _ = test_data_iterator.next()

In [ ]:
img_tensor = img_tensor[0]

In [ ]:
img_tensor = img_tensor.reshape((1,128,128,1))

In [ ]:
img_tensor.shape

In [ ]:
plt.figure(figsize=(12,6))

plt.imshow(img_tensor[0], cmap=plt.cm.binary)
plt.axis("off")
# plt.show()

In [ ]:
model.input

In [ ]:
# Extracts the outputs of the top 8 layers:
layer_outputs = [layer.output for layer in model.layers[:]]
# Creates a model that will return these outputs, given the model input:
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
activations = activation_model.predict(img_tensor)

In [ ]:
first_layer_activation = activations[0]

first_layer_activation.shape

In [ ]:
plt.figure(figsize=(12,6))

plt.matshow(first_layer_activation[0, :, :, 3], cmap='jet')
plt.axis("off")
# plt.show()

In [ ]:
activations

In [ ]:
# Names of the layers, so can have them as part of our plot
layer_names = [ layer.name for layer in model.layers[:] ]
    
images_per_row = 16

# Display feature maps
for layer_name, layer_activation in zip(layer_names[1:-2], activations[1:-2]):
    # Number of features in the feature map
    n_features = layer_activation.shape[-1]

    # The feature map has shape (1, size, size, n_features)
    size = layer_activation.shape[1]

    # Tile the activation channels matrix
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # Tile each filter into horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0, :, :, col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image

    # Display the grid
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
# plt.show()

In [ ]:
import innvestigate
import innvestigate.utils

In [ ]:
from attention_mapping import load_samples

In [ ]:
base_path = "../bin/"
list_path_to_images = [
    os.path.join(base_path, img_path) 
    for img_path in 
    list(
        filter(lambda File: File.endswith(".png"), os.listdir(base_path))
    )
]
list_path_to_images

In [ ]:
images, X = load_samples(list_path_to_images)

In [ ]:
model = load_compile_model('../model.pkl')

In [ ]:
X.shape

In [ ]:
model.predict(X).tolist()

In [ ]:
df = pd.read_csv("/home/rserage/Documents/Masters Project/Docs/Drafts/Thesis/assets/figures/adoring_knuth/proof_of_concept_data.csv")

In [ ]:
df['Entry'].tolist()

In [ ]:
df['Virus hosts'].str.split(";").apply(len)

In [ ]:
dot_img_file = "Zoon0Pred_model_architecture.png"
plot_model(model, to_file=dot_img_file, show_layer_names=False, show_shapes=False)

In [ ]:
# Strip softmax layer
# model = innvestigate.utils.model_wo #.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("deep_taylor", model)

# Add batch axis and preprocess
# x = preprocess(image[None])
# Apply analyzer w.r.t. maximum activated output-neuron
a = analyzer.analyze(X)

# Aggregate along color channels and normalize to [-1, 1]
a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
a /= np.max(np.abs(a))
# Plot
plt.imshow(a[0], cmap="seismic", clim=(-1, 1))

In [ ]:
df = pd.read_csv("~/Documents/Masters Project/Docs/Drafts/Thesis/assets/figures/adoring_knuth/Zoon0PredV/Zoon0PredV_trainingLogs.csv")

In [ ]:
df.columns

In [ ]:
df.head(12)

In [ ]:
df.sort_values("MCC", ascending=False)

In [ ]:
df.set_index('epoch', inplace=True)

In [ ]:
df['binary_accuracy'] = df['binary_accuracy'] * 100
df['val_binary_accuracy'] = df['val_binary_accuracy'] * 100

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
sns.set_context('paper')

In [ ]:
import matplotlib

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(8,5), dpi=300, sharex=True)

df[['binary_accuracy', 'val_binary_accuracy']].plot(kind='line', ax=ax1)
ax1.get_legend().remove()
# ax1.legend(None)#['Training accuracy', 'Validation accuracy'], loc='lower right', fontsize=8)
# ax1.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
# ax1.set_yticklabels(ax1.get_yticklabels(), fontsize=5)
ax1.set_ylim(50, 100)

ax1.set_ylabel('Accuracy')#, fontsize=10, fontweight='bold', rotation=0)
# ax1.yaxis.set_label_coords(-0.073, 0.9)
ax1.set_title('A', fontdict={'fontsize':10, 'fontweight':'bold'}, loc='left')

df[['loss', 'val_loss']].plot(kind='line', ax=ax2)
ax2.legend(['Training', 'Validation'], fontsize=8, bbox_to_anchor=(0.98, 1.25))
ax2.set_ylim(0.0, 2.0)
# ax2.set_yticklabels(ax2.get_yticklabels(), fontsize=5)
ax2.set_title('B', fontdict={'fontsize':10, 'fontweight':'bold'}, loc='left')

ax2.set_ylabel('Loss') #, fontsize=10, fontweight='bold', rotation=0)
# ax2.yaxis.set_label_coords(-0.075, 0.9)

# plt.subplots_adjust(wspace=0.05)

plt.savefig("/home/rserage/Documents/Masters Project/Docs/Drafts/Thesis/assets/figures/accuracy_loss.png")

In [ ]:
vals = {'': 7299,
 'KW-1160': 7299,
 'NCBI virus': 6271,
 'Virus-Host DB': 3172,
 'EID2': 3166}

In [ ]:
list(vals.values())

In [ ]:
# Bring some raw data.
frequencies = list(vals.values()) #[6, 16, 75, 160, 244, 260, 145, 73, 16, 4, 1]
# In my original code I create a series and run on that,
# so for consistency I create a series from the list.
freq_series = pd.Series(frequencies)

x_labels = list(vals.keys())

# Plot the figure.
plt.figure(figsize=(5, 3.5), dpi=300)
ax = freq_series.plot(kind="bar")
ax.get_children()[0].set_color('g')
# ax.set_title("Amount Frequency")
# ax.set_xlabel("Dataset")
ax.set_ylabel("No. of records", fontsize=7)
ax.set_xticklabels(x_labels, rotation=0, fontsize=5)
ax.set_yticklabels('', rotation=0, fontsize=7)
# ax.set_ylim(0, 8000)
ax.xaxis.grid(False)
rects = ax.patches

# Make some labels.
labels = frequencies #[f"label{i}" for i in range(len(rects))]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2, height + 5, label, ha="center", va="bottom", fontsize=7
    )

plt.savefig("/home/rserage/Documents/Masters Project/Docs/Drafts/Thesis/assets/figures/tracking.png")

In [ ]:
# import numpy as np
# import pandas as pd
# from tensorflow import keras
# import sys
# sys.path.insert(1, "../bin/")
# from model_definition import load_compile_model
from test_zoonosis_model import load_test_data

In [ ]:
physical_devices

In [ ]:
tf.__version__

In [ ]:
model = load_compile_model('../model.pkl')

In [ ]:
test = load_test_data("../Zoon0PredV/test/") # os.path.join(folder, 'test'))

steps = int(round(test.n / 2048))

In [ ]:
predictions = (model.predict(test, verbose=1)).astype(np.float32).reshape((-1,)).tolist()

In [ ]:
test.reset()
evals = model.evaluate(test, verbose=1, return_dict=True, steps=steps)

In [ ]:
test.reset()
true_values = test.labels.astype(np.float32).reshape((-1,)).tolist()

In [ ]:
test.reset()
labels = test.filenames.reshape((-1,)).tolist()

In [ ]:
# Creating dataframe automatically sorts out issue of dimentions of numpy arrays
results = pd.DataFrame.from_dict({
    'Filenames':labels,
    'True values':true_values,
    'Predictions':predictions}
)
results['Entry'] = results["Filenames"].str.split("_")[1]

In [ ]:
zoonopred_data = "/home/rserage/Documents/Masters Project/Docs/Drafts/Thesis/assets/figures/adoring_knuth/Zoon0PredV/Zoon0PredVData.csv.gz"

In [ ]:
df = pd.read_csv(zoonopred_data)

## False positives

In [ ]:
false_positives = results[(results['True values'] != results['Predictions']) & (results['Predictions'] == 1.0)].copy()
print(false_positives.shape[0])
false_positives

In [ ]:
entries_false_positives = df[df['Entry'].isin(false_positives['Entry'].tolist())].copy()

In [ ]:
entries_false_positives['Protein'].nunique()

In [ ]:
entries_false_positives['Virus hosts'].nunique()

In [ ]:
entries_false_positives['Species name'].nunique()

## False negatives

In [ ]:
false_negatives = results[(results['True values'] != results['Predictions']) & (results['Predictions'] == 0.0)].copy()
print(false_negatives.shape[0])
false_negatives

In [ ]:
entries_false_negatives = df[df['Entry'].isin(false_negatives['Entry'].tolist())].copy()

In [ ]:
entries_false_negatives['Protein'].nunique()

In [ ]:
entries_false_negatives['Virus hosts'].nunique()

In [ ]:
entries_false_negatives['Species name'].nunique()